In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  U

In [4]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping,  ModelCheckpoint, ReduceLROnPlateau, CSVLogger

ModuleNotFoundError: No module named 'distutils'

In [27]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [28]:
batch_size = 8
lr = 1e-4 
epochs = 100
height = 960
width = 1920

In [29]:
dataset_path = os.path.join("My_VineNet", "aug")

files_dir = os.path.join("files", "aug")
model_file = os.path.join(files_dir, "unet-aug.h5")
log_file = os.path.join(files_dir, "log-aug.csv")

In [30]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [31]:
create_dir(files_dir)

Building UNET

In [32]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [33]:
def encoder_block(inputs, num_filters):
   x = conv_block(inputs, num_filters)
   p = MaxPool2D((2,2))(x)
   return x, p

In [34]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

In [35]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1,128)
    s3, p3 = encoder_block(p2,256)
    s4, p4 = encoder_block(p3,512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model



In [36]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "masks", "*")))

    valid_x = sorted(glob(os.path.join(path, "valid", "images", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid", "masks", "*")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*")))
    test_y = sorted(glob(os.path.join(path, "test", "masks", "*")))

    return (train_x, train_y), (valid_x, valid_y)


In [37]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    return x

In [38]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/128.0
    x = np.expand_dims(x, axis=-1)
    return x

In [39]:
# x = cv2.imread(r"C:/Users/manta/OneDrive/Desktop/CT Scan Reconstruction/U-Net/My_VineNet/test/masks/Block_5E1_Row_1_Middle_3304_instanceIds.png", cv2.IMREAD_GRAYSCALE)
# x = x/128.0
# window_name = 'image'
# cv2.imshow(window_name, x)
# cv2.waitKey(1)
# print(np.unique(x))

In [40]:
def tf_parse(x, y):
    def _parse(x,y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])

    return x, y

In [41]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [42]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 2000 - 2000
Valid: 119 - 119


In [43]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [44]:
for x, y in valid_dataset:
    print (x.shape, y.shape)

(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(8, 960, 1920, 3) (8, 960, 1920, 1)
(7, 960, 1920, 3) (7, 960, 1920, 1)


In [45]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [46]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 960, 1920,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 960, 1920, 6  1792        ['input_2[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization_18 (BatchN  (None, 960, 1920, 6  256        ['conv2d_19[0][0]']              
 ormalization)                  4)                                                             

In [47]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['acc'])

In [48]:
callbacks = [
        ModelCheckpoint(model_file, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=4),
        CSVLogger(log_file),
        EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
]

In [49]:
 model.fit(
     train_dataset,
     validation_data=valid_dataset,
     epochs=1,
     callbacks=callbacks
 )

In [50]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
